In [1]:
import pandas as pd
import numpy as np
import torch
# import torch.nn as nn
# import torch.optim as optim
# from torchvision import models, transforms
# import matplotlib.pyplot as plt
# from sklearn.metrics import precision_recall_curve, auc
# import numpy as np
# import matplotlib.pyplot as plt
from tqdm.auto import tqdm
# from sklearn.metrics import f1_score
from sentence_transformers import SentenceTransformer, SimilarityFunction
# from sklearn.metrics import roc_auc_score
import re
# import os
# import warnings
# warnings.filterwarnings("ignore")

In [2]:
tqdm.pandas()

# Загружаем данные 

In [3]:
%%time
path = "datasets/" # Ваш путь до директории с данными /path/to/data/
data = pd.read_csv(path + 'train_events.csv')
video = pd.read_csv(path + 'video_info_v2.csv')
targets = pd.read_csv(path + 'train_targets.csv')
all_events = pd.read_csv(path + 'all_events.csv')

# заполянем пропуски самыми частотными
freq_ua_client_type = all_events['ua_client_type'].value_counts().index[0]
freq_ua_client_name = all_events['ua_client_name'].value_counts().index[0]
all_events['ua_client_type'] = all_events['ua_client_type'].fillna(freq_ua_client_type)
all_events['ua_client_name'] = all_events['ua_client_name'].fillna(freq_ua_client_name)
all_events['ua_os'] = all_events['ua_os'].fillna('vpn')
data['ua_os'] = data['ua_os'].fillna('vpn')

# Преобразование даты и время в тип даты
all_events['event_timestamp'] = pd.to_datetime(all_events['event_timestamp'])
data['event_timestamp'] = pd.to_datetime(data['event_timestamp'])

# продолжительность видео duration - указан в миллисекнудах, а просмотр человеком (total_watchtime) в секндах приводим к одному виду
# video['duration'] = video['duration']/1000
all_events['total_watchtime'] = all_events['total_watchtime'] * 1000
data['total_watchtime'] = data['total_watchtime'] * 1000

data.shape, video.shape, targets.shape, all_events.shape

CPU times: total: 56.9 s
Wall time: 56.8 s


((1759616, 9), (481480, 5), (180012, 4), (8439624, 9))

In [4]:
%%time
from sklearn.model_selection import train_test_split

train_users_df, test_users_df = train_test_split(targets, stratify=(targets[['age_class', 'sex']]), test_size=0.1)
# train_users_df.shape, test_users_df.shape

data_video = data.merge(video, on='rutube_video_id')
train_users_df = data_video.merge(train_users_df, on='viewer_uid')
test_users_df = data_video.merge(test_users_df, on='viewer_uid')
del data_video

train_users_df.shape, test_users_df.shape

CPU times: total: 1.66 s
Wall time: 1.66 s


((1582553, 16), (177063, 16))

In [5]:
test_users_df.groupby(by='viewer_uid')[['category', 'title']]

In [6]:
%%time
category_test_users_df = test_users_df.groupby(by='viewer_uid')['category'].agg(lambda col: ' '.join(col))
title_test_users_df = test_users_df.groupby(by='viewer_uid')['title'].agg(lambda col: ' '.join(col))

category_train_users_df = train_users_df.groupby(by='viewer_uid')['category'].agg(lambda col: ' '.join(col))
title_train_users_df = train_users_df.groupby(by='viewer_uid')['title'].agg(lambda col: ' '.join(col))


category_test_users_df.shape, title_test_users_df.shape, category_train_users_df.shape, title_train_users_df.shape

CPU times: total: 3.72 s
Wall time: 3.72 s


((18002,), (18002,), (162010,), (162010,))

In [7]:
%%time
category_test_users_df = category_test_users_df.reset_index().merge(targets, on='viewer_uid')
title_test_users_df = title_test_users_df.reset_index().merge(targets, on='viewer_uid')
category_train_users_df = category_train_users_df.reset_index().merge(targets, on='viewer_uid')
title_train_users_df = title_train_users_df.reset_index().merge(targets, on='viewer_uid')

category_test_users_df.shape, title_test_users_df.shape, category_train_users_df.shape, title_train_users_df.shape

CPU times: total: 109 ms
Wall time: 96.3 ms


((18002, 5), (18002, 5), (162010, 5), (162010, 5))

In [8]:
# Функция для чистки текста
def clean_text(text):
    if pd.isna(text):
        return ''
    # оставляем только слова и цифры
    pattern = r"[a-zA-Z]+|[\d]+|[а-яА-Я]+"
    text = " ".join(re.findall(pattern, text.lower()))
    return text

In [9]:
%%time
category_test_users_df['clear_category'] = category_test_users_df['category'].apply(lambda x: clean_text(x))
category_train_users_df['clear_category'] = category_train_users_df['category'].apply(lambda x: clean_text(x))
title_test_users_df['clear_title'] = title_test_users_df['title'].apply(lambda x: clean_text(x))
title_train_users_df['clear_title'] = title_train_users_df['title'].apply(lambda x: clean_text(x))

category_test_users_df.shape, title_test_users_df.shape, category_train_users_df.shape, title_train_users_df.shape

CPU times: total: 4.31 s
Wall time: 4.32 s


((18002, 6), (18002, 6), (162010, 6), (162010, 6))

In [10]:
category_test_users_df['sex_class'] = np.where(category_test_users_df['sex'] == 'male', 1, 0)
category_train_users_df['sex_class'] = np.where(category_train_users_df['sex'] == 'male', 1, 0)
title_test_users_df['sex_class'] = np.where(title_test_users_df['sex'] == 'male', 1, 0)
title_train_users_df['sex_class'] = np.where(title_train_users_df['sex'] == 'male', 1, 0)

category_test_users_df.shape, title_test_users_df.shape, category_train_users_df.shape, title_train_users_df.shape

((18002, 7), (18002, 7), (162010, 7), (162010, 7))

In [11]:
# encode_model = SentenceTransformer('FacebookAI/xlm-roberta-large')
encode_model = SentenceTransformer('sentence-transformers/LaBSE')
encode_model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Dense({'in_features': 768, 'out_features': 768, 'bias': True, 'activation_function': 'torch.nn.modules.activation.Tanh'})
  (3): Normalize()
)

# Формируем эмбеддинги

In [12]:
%%time
# Wall time: 1h 25min 26s
# Wall time: 29min 12s
title_test_users_df['embeddings'] = title_test_users_df['clear_title'].progress_apply(encode_model.encode)
title_train_users_df['embeddings'] = title_train_users_df['clear_title'].progress_apply(encode_model.encode)

title_test_users_df.shape, title_train_users_df.shape

  0%|          | 0/18002 [00:00<?, ?it/s]

  0%|          | 0/162010 [00:00<?, ?it/s]

CPU times: total: 29min 13s
Wall time: 29min 12s


((18002, 8), (162010, 8))

In [17]:
# %%time
# title_test_users_df.to_parquet('title_test_users_df_LABSE.parquet')
# title_train_users_df.to_parquet('title_train_users_df_LABSE.parquet')
# title_test_users_df.shape, title_train_users_df.shape

In [14]:
union_title_users_df = pd.concat([title_test_users_df, title_train_users_df])
union_title_users_df.shape

(180012, 8)

In [15]:
%%time
# Wall time: 57min 45s
category_test_users_df['embeddings'] = category_test_users_df['clear_category'].progress_apply(encode_model.encode)
category_train_users_df['embeddings'] = category_train_users_df['clear_category'].progress_apply(encode_model.encode)

category_test_users_df.shape, category_train_users_df.shape

  0%|          | 0/18002 [00:00<?, ?it/s]

  0%|          | 0/162010 [00:00<?, ?it/s]

In [18]:
# %%time
# category_test_users_df.to_parquet('category_test_users_df_LABSE.parquet')
# category_train_users_df.to_parquet('category_train_users_df_LABSE.parquet')
# category_test_users_df.shape, category_train_users_df.shape

In [20]:
union_category_users_df = pd.concat([category_test_users_df, category_train_users_df])
union_category_users_df.shape

(180012, 8)

# Тестовые данные для сабмита

In [21]:
# submit_test_users_df = pd.read_csv('Dima/sample_10000.csv')
submit_test_users_df = pd.read_csv('test_dataset/test_events.csv')
submit_test_users_df.shape


(587735, 9)

In [22]:
submit_test_users_df = submit_test_users_df.merge(video, on='rutube_video_id')
submit_test_users_df.shape

(587735, 13)

In [23]:
# data_video = all_events.merge(video, on='rutube_video_id')
# submit_test_users_df = data_video.merge(submit_test_users_df, on='viewer_uid')
# submit_test_users_df.shape

In [24]:
category_submit_test_users_df = submit_test_users_df.groupby(by='viewer_uid')['category'].agg(lambda col: ' '.join(col))
title_submit_test_users_df = submit_test_users_df.groupby(by='viewer_uid')['title'].agg(lambda col: ' '.join(col))
category_submit_test_users_df = pd.DataFrame(category_submit_test_users_df)
title_submit_test_users_df = pd.DataFrame(title_submit_test_users_df)

category_submit_test_users_df.shape,  title_submit_test_users_df.shape

((60004, 1), (60004, 1))

In [25]:
category_submit_test_users_df['clear_category'] = category_submit_test_users_df['category'].apply(lambda x: clean_text(x))
title_submit_test_users_df['clear_title'] = title_submit_test_users_df['title'].apply(lambda x: clean_text(x))

category_submit_test_users_df.shape,  title_submit_test_users_df.shape

((60004, 2), (60004, 2))

In [26]:
%%time
title_submit_test_users_df['embeddings'] = title_submit_test_users_df['clear_title'].progress_apply(encode_model.encode)
category_submit_test_users_df['embeddings'] = category_submit_test_users_df['clear_category'].progress_apply(encode_model.encode)

category_submit_test_users_df.shape, title_submit_test_users_df.shape

  0%|          | 0/60004 [00:00<?, ?it/s]

  0%|          | 0/60004 [00:00<?, ?it/s]

CPU times: total: 18min 5s
Wall time: 18min 9s


((60004, 3), (60004, 3))

# Logistic Regression

# Генерируем фичи на базе title

### Берем отложенную выборку для локлаьного тесте

In [27]:
train_users_df = pd.read_csv('train_viewer_uid_v2.csv')
test_users_df = pd.read_csv('test_viewer_uid_v2.csv')

train_users_df.shape, test_users_df.shape

((1505791, 2), (170274, 2))

In [28]:
TRAIN_title_users_df = union_title_users_df[union_title_users_df['viewer_uid'].isin(train_users_df['viewer_uid'])]
TEST_title_users_df = union_title_users_df[union_title_users_df['viewer_uid'].isin(test_users_df['viewer_uid'])]

TRAIN_title_users_df.shape, TEST_title_users_df.shape

((155197, 8), (17221, 8))

In [29]:
TRAIN_title_users_df = union_title_users_df.copy()
TEST_title_users_df = title_submit_test_users_df.reset_index().copy()

TRAIN_title_users_df.shape, TEST_title_users_df.shape

((180012, 8), (60004, 4))

In [30]:
# submit_test_users_df
# set(submit_test_users_df.columns) - set(TRAIN_title_users_df.columns), set(TRAIN_title_users_df.columns) -set(submit_test_users_df.columns)
# TRAIN_title_users_df

# Генерируем фичи через кроссвалидацию на базе названий просмотрренных видео

In [33]:
%%time
# Генерируем фичи через кроссвалидацию на базе названий просмотрренных видео
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import metrics

def calc_cv_target(TRAIN_title_users_df:pd.DataFrame, TEST_title_users_df:pd.DataFrame, target_col:str = 'sex_class'):
    
    X = TRAIN_title_users_df['embeddings'].values
    y = TRAIN_title_users_df[target_col].values
    TEST_X = TEST_title_users_df['embeddings'].values

    # Количество фолдов для кросс-валидации
    n_folds = 5

    # Создаем объект StratifiedKFold для кросс-валидации
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=53)

    # Создаем массив для хранения OOF предсказаний
    oof_predictions = np.zeros(len(X))

    # Модель логистической регрессии
    model = LogisticRegression()

    # Проходим по каждому фолду
    for i, (train_idx, valid_idx) in enumerate(tqdm(skf.split(X, y))):
        X_train, X_valid = X[train_idx], X[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        # Обучаем модель на данных фолда
        model.fit(list(X_train), list(y_train))

        # Предсказываем вероятности для валидационных данных (берем вероятность класса 1)
        predicted_proba = model.predict_proba(list(X_valid))[:, 1]
        predicted = model.predict(list(X_valid))
        oof_predictions[valid_idx] = predicted_proba
        TEST_title_users_df[f'logistic_oof_{target_col}_{i}'] = model.predict_proba(list(TEST_X))[:, 1]
        print("Logistic Regression Accuracy:",metrics.accuracy_score(y_valid, predicted))
        print("Logistic Regression Precision:",metrics.precision_score(y_valid, predicted))
        print("Logistic Regression Recall:",metrics.recall_score(y_valid, predicted))
        print("Logistic Regression f1_score:",metrics.f1_score(y_valid, predicted))    

    # Теперь oof_predictions содержит предсказания для всех данных, которые можно использовать как новую фичу
    TRAIN_title_users_df[f'logistic_oof_{target_col}'] = oof_predictions

    temp_cols = [f'logistic_oof_{target_col}_{i}' for i in range(n_folds)]
    TEST_title_users_df[f'logistic_oof_{target_col}'] = TEST_title_users_df[temp_cols].mean(axis=1)
    TEST_title_users_df = TEST_title_users_df.drop(columns=temp_cols, errors='ignore')

    return TRAIN_title_users_df, TEST_title_users_df 

TRAIN_title_users_df, TEST_title_users_df = calc_cv_target(TRAIN_title_users_df, TEST_title_users_df, target_col='sex_class')
TRAIN_title_users_df.shape, TEST_title_users_df.shape

0it [00:00, ?it/s]

Logistic Regression Accuracy: 0.7337999611143515
Logistic Regression Precision: 0.724344249158609
Logistic Regression Recall: 0.7539898793304788
Logistic Regression f1_score: 0.7388698163587816
Logistic Regression Accuracy: 0.7371330166930533
Logistic Regression Precision: 0.7271854498906608
Logistic Regression Recall: 0.7581604849024078
Logistic Regression f1_score: 0.7423499945551563
Logistic Regression Accuracy: 0.7282928726181879
Logistic Regression Precision: 0.7176139680517964
Logistic Regression Recall: 0.7519323805816605
Logistic Regression f1_score: 0.7343724542442839
Logistic Regression Accuracy: 0.7317926781845453
Logistic Regression Precision: 0.7189817493294062
Logistic Regression Recall: 0.7601623755769338
Logistic Regression f1_score: 0.7389988106822359
Logistic Regression Accuracy: 0.739181156602411
Logistic Regression Precision: 0.7305463131909413
Logistic Regression Recall: 0.7570348125903682
Logistic Regression f1_score: 0.7435547301726021
CPU times: total: 2min 8s
W

((180012, 9), (60004, 6))

In [34]:
%%time
# Генерируем фичи через кроссвалидацию на базе названий просмотрренных видео
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression

def calc_cv_target_regr(TRAIN_title_users_df:pd.DataFrame, TEST_title_users_df:pd.DataFrame, target_col:str = 'age'):
    
    X = TRAIN_title_users_df['embeddings'].values
    y = TRAIN_title_users_df[target_col].values
    TEST_X = TEST_title_users_df['embeddings'].values

    # Количество фолдов для кросс-валидации
    n_folds = 5

    # Создаем объект StratifiedKFold для кросс-валидации
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=53)

    # Создаем массив для хранения OOF предсказаний
    oof_predictions = np.zeros(len(X))

    # Модель линейной  регрессии
    model = LinearRegression()

    # Проходим по каждому фолду
    for i, (train_idx, valid_idx) in enumerate(tqdm(skf.split(X, y))):
        X_train, X_valid = X[train_idx], X[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        # Обучаем модель на данных фолда
        model.fit(list(X_train), list(y_train))

        # Предсказываем возраст
        predicted = model.predict(list(X_valid))
        oof_predictions[valid_idx] = predicted
        TEST_title_users_df[f'linear_oof_{target_col}_{i}'] = model.predict(list(TEST_X))
        print("Logistic Regression MAE:",metrics.mean_absolute_error(y_valid, predicted))

    # Теперь oof_predictions содержит предсказания для всех данных, которые можно использовать как новую фичу
    TRAIN_title_users_df[f'linear_oof_{target_col}'] = oof_predictions

    temp_cols = [f'linear_oof_{target_col}_{i}' for i in range(n_folds)]
    TEST_title_users_df[f'linear_oof_{target_col}'] = TEST_title_users_df[temp_cols].mean(axis=1)
    TEST_title_users_df = TEST_title_users_df.drop(columns=temp_cols, errors='ignore')

    return TRAIN_title_users_df, TEST_title_users_df 

TRAIN_title_users_df, TEST_title_users_df = calc_cv_target_regr(TRAIN_title_users_df, TEST_title_users_df, target_col='age')
TRAIN_title_users_df.shape, TEST_title_users_df.shape

0it [00:00, ?it/s]

Logistic Regression MAE: 6.7321050521172054
Logistic Regression MAE: 6.744154309986717
Logistic Regression MAE: 6.736283965073959
Logistic Regression MAE: 6.7419337161441195
Logistic Regression MAE: 6.760596948109443
CPU times: total: 44.9 s
Wall time: 15.7 s


((180012, 10), (60004, 6))

In [35]:
%%time
# Сохраняем фичи полученные через кросс валидацию
TRAIN_title_users_df[['viewer_uid', 'logistic_oof_sex_class', 'linear_oof_age']].to_csv(f'cv_TRAIN_title_users_df_v3.csv')
TEST_title_users_df[['viewer_uid', 'logistic_oof_sex_class', 'linear_oof_age']].to_csv(f'cv_TEST_title_users_df_v3.csv')


CPU times: total: 469 ms
Wall time: 468 ms


In [36]:
TRAIN_title_users_df.shape, TEST_title_users_df.shape

((180012, 10), (60004, 6))

# Генерируем фичи на базе category

In [37]:
TRAIN_category_users_df = union_category_users_df[union_category_users_df['viewer_uid'].isin(train_users_df['viewer_uid'])]
TEST_category_users_df = union_category_users_df[union_category_users_df['viewer_uid'].isin(test_users_df['viewer_uid'])]

TRAIN_category_users_df.shape, TEST_category_users_df.shape

((155197, 8), (17221, 8))

In [38]:
TRAIN_category_users_df = union_category_users_df.copy()
TEST_category_users_df = category_submit_test_users_df.reset_index().copy()

TRAIN_category_users_df.shape, TEST_category_users_df.shape

((180012, 8), (60004, 4))

In [39]:
%%time
# Генерируем фичи через кроссвалидацию на базе названий просмотрренных видео
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

def calc_cv_target(TRAIN_category_users_df:pd.DataFrame, TEST_category_users_df:pd.DataFrame, target_col:str = 'sex_class'):
    
    X = TRAIN_category_users_df['embeddings'].values
    y = TRAIN_category_users_df[target_col].values
    TEST_X = TEST_category_users_df['embeddings'].values

    # Количество фолдов для кросс-валидации
    n_folds = 5

    # Создаем объект StratifiedKFold для кросс-валидации
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=53)

    # Создаем массив для хранения OOF предсказаний
    oof_predictions = np.zeros(len(X))

    # Модель логистической регрессии
    model = LogisticRegression()

    # Проходим по каждому фолду
    for i, (train_idx, valid_idx) in enumerate(tqdm(skf.split(X, y))):
        X_train, X_valid = X[train_idx], X[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        # Обучаем модель на данных фолда
        model.fit(list(X_train), list(y_train))

        # Предсказываем вероятности для валидационных данных (берем вероятность класса 1)
        predicted_proba = model.predict_proba(list(X_valid))[:, 1]
        predicted = model.predict(list(X_valid))
        oof_predictions[valid_idx] = predicted_proba
        TEST_category_users_df[f'logistic_oof_cat_{target_col}_{i}'] = model.predict_proba(list(TEST_X))[:, 1]
        print("Logistic Regression Accuracy:",metrics.accuracy_score(y_valid, predicted))
        print("Logistic Regression Precision:",metrics.precision_score(y_valid, predicted))
        print("Logistic Regression Recall:",metrics.recall_score(y_valid, predicted))
        print("Logistic Regression f1_score:",metrics.f1_score(y_valid, predicted))    

    # Теперь oof_predictions содержит предсказания для всех данных, которые можно использовать как новую фичу
    TRAIN_category_users_df[f'logistic_oof_cat_{target_col}'] = oof_predictions

    temp_cols = [f'logistic_oof_cat_{target_col}_{i}' for i in range(n_folds)]
    TEST_category_users_df[f'logistic_oof_cat_{target_col}'] = TEST_category_users_df[temp_cols].mean(axis=1)
    TEST_category_users_df = TEST_category_users_df.drop(columns=temp_cols, errors='ignore')

    return TRAIN_category_users_df, TEST_category_users_df 

TRAIN_category_users_df, TEST_category_users_df = calc_cv_target(TRAIN_category_users_df, TEST_category_users_df, target_col='sex_class')
TRAIN_category_users_df.shape, TEST_category_users_df.shape

0it [00:00, ?it/s]

Logistic Regression Accuracy: 0.7139405049579202
Logistic Regression Precision: 0.7006266318537859
Logistic Regression Recall: 0.7460935327809598
Logistic Regression f1_score: 0.7226456251851453
Logistic Regression Accuracy: 0.7168847040524401
Logistic Regression Precision: 0.7046553173602353
Logistic Regression Recall: 0.7457598843352055
Logistic Regression f1_score: 0.7246251519654194
Logistic Regression Accuracy: 0.7118493417032387
Logistic Regression Precision: 0.6967471686404303
Logistic Regression Recall: 0.7492075849413334
Logistic Regression f1_score: 0.7220257234726688
Logistic Regression Accuracy: 0.7156269096161324
Logistic Regression Precision: 0.7021243279920664
Logistic Regression Recall: 0.7480398153811934
Logistic Regression f1_score: 0.7243551774271714
Logistic Regression Accuracy: 0.715515804677518
Logistic Regression Precision: 0.7030217186024551
Logistic Regression Recall: 0.7452452452452453
Logistic Regression f1_score: 0.7235179786200194
CPU times: total: 2min 49s

((180012, 9), (60004, 5))

In [40]:
%%time
# Генерируем фичи через кроссвалидацию на базе названий просмотрренных видео
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression

def calc_cv_target_regr(TRAIN_category_users_df:pd.DataFrame, TEST_category_users_df:pd.DataFrame, target_col:str = 'age'):
    
    X = TRAIN_category_users_df['embeddings'].values
    y = TRAIN_category_users_df[target_col].values
    TEST_X = TEST_category_users_df['embeddings'].values

    # Количество фолдов для кросс-валидации
    n_folds = 5

    # Создаем объект StratifiedKFold для кросс-валидации
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=53)

    # Создаем массив для хранения OOF предсказаний
    oof_predictions = np.zeros(len(X))

    # Модель линейной  регрессии
    model = LinearRegression()

    # Проходим по каждому фолду
    for i, (train_idx, valid_idx) in enumerate(tqdm(skf.split(X, y))):
        X_train, X_valid = X[train_idx], X[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        # Обучаем модель на данных фолда
        model.fit(list(X_train), list(y_train))

        # Предсказываем возраст
        predicted = model.predict(list(X_valid))
        oof_predictions[valid_idx] = predicted
        TEST_category_users_df[f'linear_oof_cat_{target_col}_{i}'] = model.predict(list(TEST_X))
        print("Logistic Regression MAE:",metrics.mean_absolute_error(y_valid, predicted))

    # Теперь oof_predictions содержит предсказания для всех данных, которые можно использовать как новую фичу
    TRAIN_category_users_df[f'linear_oof_cat_{target_col}'] = oof_predictions

    temp_cols = [f'linear_oof_cat_{target_col}_{i}' for i in range(n_folds)]
    TEST_category_users_df[f'linear_oof_cat_{target_col}'] = TEST_category_users_df[temp_cols].mean(axis=1)
    TEST_category_users_df = TEST_category_users_df.drop(columns=temp_cols, errors='ignore')

    return TRAIN_category_users_df, TEST_category_users_df 

TRAIN_category_users_df, TEST_category_users_df = calc_cv_target_regr(TRAIN_category_users_df, TEST_category_users_df, target_col='age')
TRAIN_category_users_df.shape, TEST_category_users_df.shape

0it [00:00, ?it/s]

Logistic Regression MAE: 6.925046903929525
Logistic Regression MAE: 6.928785271119188
Logistic Regression MAE: 6.925280184449371
Logistic Regression MAE: 6.934120755693355
Logistic Regression MAE: 6.938672112992099
CPU times: total: 43.8 s
Wall time: 15 s


((180012, 10), (60004, 6))

In [41]:
%%time
# Сохраняем фичи полученные через кросс валидацию
TRAIN_category_users_df[['viewer_uid', 'logistic_oof_cat_sex_class', 'linear_oof_cat_age']].to_csv(f'cv_TRAIN_category_users_df_v3.csv')
TEST_category_users_df[['viewer_uid', 'logistic_oof_cat_sex_class', 'linear_oof_cat_age']].to_csv(f'cv_TEST_category_users_df_v3.csv')

TRAIN_category_users_df.shape, TEST_category_users_df.shape 

CPU times: total: 469 ms
Wall time: 465 ms


((180012, 10), (60004, 6))